# Random Forest Regression - Sem variáveis adquiridas após o abate

In [8]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix
from sklearn import metrics
from sklearn.metrics import mean_squared_error

In [9]:
df = pd.read_csv('../../dados/dadosTransformados_v1.csv', sep=';')

In [10]:
df.shape

(960939, 53)

In [11]:
pd.set_option('display.max_columns',999)
pd.set_option("display.max_rows", 999)

In [12]:
df.sample(5)

,tipificacao_1,tipificacao_2,tipificacao_3,maturidade,acabamento,peso,classificacao_estabelecimento,outros_incentivos,pratica_recuperacao_pastagem,fertiirrigacao,ilp,ilpf,ifp,fabrica_racao,identificacao_individual,regua_de_manejo,bpa,participa_aliancas_mercadologicas,rastreamento_sisbov,tot1m_chuva,med1m_formituinst,med1m_ndvi,med1m_prer_soja,med1m_prer_milho,med1m_prer_boi,tot3m_chuva,med3m_formituinst,med3m_ndvi,med3m_prer_soja,med3m_prer_milho,med3m_prer_boi,tot6m_chuva,med6m_formituinst,med6m_ndvi,med6m_prer_soja,med6m_prer_milho,med6m_prer_boi,tot12m_chuva,med12m_formituinst,med12m_ndvi,med12m_prer_soja,med12m_prer_milho,med12m_prer_boi,cnt1m_cl_ituinst,cnt3m_cl_ituinst,cnt6m_cl_ituinst,cnt12m_cl_ituinst,ano,categoria,classificacao,longitude,latitude,mes
510786,1,0,0,2,3,306.0,26.0,0,1,1,0,0,0,1,1,1,0,0,1,11.38,67.88,0.47,85.41,42.88,139.13,120.76,71.22,0.53,84.80,41.45,141.32,367.89,72.28,0.57,79.87,38.52,143.49,609.58,70.88,0.52,75.70,33.72,140.05,0.8621,0.6782,0.5878,0.6589,2018,2,1,-53.76168,-20.44368,6
28981,0,0,1,2,3,206.2,21.0,1,1,0,0,0,1,0,0,1,0,0,0,155.79,74.49,0.65,68.86,27.71,135.87,343.08,74.74,0.65,68.42,30.06,138.81,641.07,74.67,0.65,70.97,32.36,141.91,368.42,75.96,0.65,70.97,32.36,141.91,0.3226,0.3118,0.2632,0.2632,2017,2,1,-54.55245,-18.10186,6
99277,1,0,0,2,3,349.6,26.0,0,1,0,1,0,0,1,1,1,0,0,1,78.90,69.60,0.39,69.92,26.58,132.95,146.79,68.24,0.49,70.25,26.58,128.99,439.43,69.25,0.56,69.40,28.33,133.87,673.70,70.00,0.58,70.75,30.18,136.98,0.8387,0.8495,0.8087,0.7355,2017,2,1,-55.15011,-22.55058,8
245141,0,1,0,2,2,278.2,26.0,0,1,0,1,0,0,1,1,1,0,0,1,238.26,71.69,0.63,73.74,31.84,140.75,421.82,71.88,0.54,71.77,30.54,141.74,741.53,69.65,0.53,71.04,28.55,135.13,1661.02,70.29,0.59,71.04,30.31,138.25,0.8065,0.7527,0.8087,0.7387,2017,4,1,-54.19884,-23.06000,11
527202,0,0,1,2,3,204.0,30.0,0,1,0,0,1,0,1,1,1,0,1,0,66.58,70.81,0.58,84.83,40.55,138.53,222.58,72.09,0.62,85.49,41.05,140.76,788.67,73.99,0.65,80.47,38.71,143.10,1623.76,75.04,0.60,76.20,34.09,140.48,0.7097,0.5484,0.3880,0.2995,2018,2,1,-55.80457,-20.48236,7


## Criando conjunto de treino e de teste

In [13]:
y = df['peso']
# Retirar a variável alvo e as variáveis obtidas após o abate 
X = df.drop(['peso','acabamento', 'maturidade', '=='],axis=1)

In [14]:
SEED = 43

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=SEED)

In [16]:
X_train.shape

(672657, 49)

## Treinando o Modelo

In [17]:
model = RandomForestRegressor(n_estimators = 100, random_state = 0)
model.fit(X, y)  

RandomForestRegressor(random_state=0)

## Previsões dados de treino

In [18]:
y_predicted_train = model.predict(X_train)
y_predicted = model.predict(X_test)

## Analisando o Modelo

In [19]:
print(model.score(X_test, y_test))

0.7965032503926766


In [20]:
print(np.sqrt(mean_squared_error(y_test, y_predicted)))

19.991932201321678


In [21]:
print('Mean Absolute Error (MAE):', metrics.mean_absolute_error(y_test, y_predicted))
print('Mean Squared Error (MSE):', metrics.mean_squared_error(y_test, y_predicted))
print('Root Mean Squared Error (RMSEp):', np.sqrt(metrics.mean_squared_error(y_test, y_predicted)))

Mean Absolute Error (MAE): 15.107740889922992
Mean Squared Error (MSE): 399.67735314224257
Root Mean Squared Error (RMSEp): 19.991932201321678


## Validação do Modelo

In [22]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate

In [23]:
SEED = 1580
np.random.seed(SEED)
cv = KFold(n_splits = 10, shuffle = True)
results = cross_validate(model, X, y,cv=cv, return_train_score= False)

In [24]:
def imprime_resulado(results) :
    media = results ['test_score'].mean()
    print('Accuracy médio %.2f' % (media*100))
    desvio_padrao = results['test_score'].std()
    print("Accuracy intervalo [ %.2f, %.2f ]" % (media - 2 * desvio_padrao, media + 2 * desvio_padrao))
    

In [25]:
imprime_resulado(results)

Accuracy médio 78.53
Accuracy intervalo [ 0.78, 0.79 ]


## GridSearchCV

In [ ]:
from sklearn.model_selection import GridSearchCV